In [1]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-tq1hwq6j
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-tq1hwq6j
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 665kB 38.0MB/s 
     |████████████████████████████████| 256kB 53.9MB/s 
     |████████████████████████████████| 983kB 53.7MB/s 
     |████████████████████████████████| 798kB 49.6MB/s 
     |████████████████████████████████| 890kB 52.4MB/s 
     |████████████████████████████████| 1.1MB 53.6MB/s 
     |████████████████████████████████| 3.8MB 53.1MB/s 
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=148939 sha256=f4f0e1fe34502abebcce3a48f0b4492203af043f12c892e38b29b2a057080173
  Stored in directory: /tmp/pip-ephem-wheel-cache-e59q__h5/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Creat

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#data_folder = "./drive/My Drive/capstone/data/"
data_folder = "/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [6]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

In [29]:
benchmark.head()

,label,text
163,0,"At the moment , there are approximately 20 Via..."
109,0,"Among other industrial stocks , Metso added 0...."
252,-1,Net sales of the Paper segment decreased to EU...
844,0,"Sami Sepp+Ænen , CEO for Elisa Eesti , says th..."
1034,-1,Tesla settles in cash $920 million convertible...


In [0]:

# benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', index = False, header = False)
# benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', index = False, header = False)

In [17]:
pd.read_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/dev.csv")

,0,- Cash flow from operating activities before investments was EUR 0.8 -1.2 million .
0,1,Complex Mega-Companies Dominate South Korea's ...
1,-1,Dealers said the share was largely hit by inve...
2,1,Industrial stocks: A big bet on economic growt...
3,-1,"Operating profit , excluding non-recurring ite..."
4,1,Net sales rose by 25.5 % year-on-year to EUR59...
...,...,...
141,-1,"Finnish Scanfil , a systems supplier and contr..."
142,0,The repurchase of the bonds was executed in th...
143,1,Kosovo's Economy Still Struggling Five Years A...
144,-1,Will the Internet Destroy the Stock Market?. T...


#### Build corpus

In [33]:
#corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
                                         train_file='train.csv', ## passing in file names manuelly when it can't auto detect
                                         dev_file='dev.csv',
                                         test_file='test.csv'
)

2020-05-25 23:35:21,378 Reading data from /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training
2020-05-25 23:35:21,385 Train: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/train.csv
2020-05-25 23:35:21,387 Dev: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/dev.csv
2020-05-25 23:35:21,388 Test: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/test.csv


#### Create word embeddings

In [19]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

# word_embeddings = [BertEmbeddings()]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.



2020-05-25 23:21:12,931 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpxc7o3pm5


100%|██████████| 19689779/19689779 [00:04<00:00, 4773033.95B/s]

2020-05-25 23:21:18,268 copying /tmp/tmpxc7o3pm5 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2020-05-25 23:21:18,291 removing temp file /tmp/tmpxc7o3pm5


2020-05-25 23:21:36,154 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpo8i3uz3h


100%|██████████| 19689779/19689779 [00:04<00:00, 4711701.54B/s]

2020-05-25 23:21:41,570 copying /tmp/tmpo8i3uz3h to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2020-05-25 23:21:41,593 removing temp file /tmp/tmpo8i3uz3h


#### First Stage Fine-tuning

In [34]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

2020-05-25 23:35:34,627 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.
100%|██████████| 1314/1314 [00:01<00:00, 1196.07it/s]

2020-05-25 23:35:35,971 [b'0', b'-1', b'1']
2020-05-25 23:35:35,985 ----------------------------------------------------------------------------------------------------


2020-05-25 23:35:35,992 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
    

{'dev_loss_history': [1.1823023557662964,
  0.8175520300865173,
  0.7285744547843933,
  0.8591181039810181,
  0.8208729028701782,
  0.9078011512756348,
  1.0128289461135864,
  0.8733577728271484,
  0.8282421231269836,
  0.8999382257461548],
 'dev_score_history': [0.6689342403628118,
  0.7414965986394558,
  0.7732426303854876,
  0.764172335600907,
  0.7551020408163265,
  0.7097505668934241,
  0.7596371882086168,
  0.782312925170068,
  0.7777777777777778,
  0.7913832199546486],
 'test_score': 0.726027397260274,
 'train_loss_history': [1.1205680032034178,
  0.899281778851071,
  0.8085721360670554,
  0.7352465584471419,
  0.7515609385194005,
  0.6912054330916018,
  0.6460504217727764,
  0.48869013544675466,
  0.4667409747033506,
  0.5006147099507822]}

In [36]:
from flair.data import Sentence
# create example sentence
sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

# predict class and print
classifier.predict(sentence)

print(sentence.labels)

[-1 (0.9964)]


In [37]:
#  access the Sentence objects in each split directly
print(corpus.test[0])

Sentence: "Profit before taxes was EUR 5.4 mn , up from EUR 3.6 mn a year earlier ."   [− Tokens: 17  − Sentence-Labels: {'class': [1 (1.0)]}]


In [38]:
print(corpus.train[0])

Sentence: "At the moment , there are approximately 20 Vianor sales offices in Russia ."   [− Tokens: 14  − Sentence-Labels: {'class': [0 (1.0)]}]


In [39]:
#outputs detailed information on the dataset, each split, and the distribution of class labels.
stats = corpus.obtain_statistics()
print(stats)

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1168,
        "number_of_documents_per_class": {
            "0": 389,
            "-1": 399,
            "1": 380
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 36665,
            "min": 4,
            "max": 68,
            "avg": 31.391267123287673
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 146,
        "number_of_documents_per_class": {
            "1": 59,
            "0": 48,
            "-1": 39
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 4522,
            "min": 8,
            "max": 69,
            "avg": 30.972602739726028
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 147,
        "number_of_documents_per_class": {
            "-1": 56,
            "1": 46,
            "0": 45
        },

### Second Stage (train on hand annotated datasets)

#### Build corpus

In [40]:
#new_data_folder = './drive/My Drive/capstone/data/second_stage/'
new_data_folder = '/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/'
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
                                         train_file='GDP_train_df_oversampled.csv',
                                         dev_file = 'GDP_eva_df_oversampled.csv',
                                         test_file = 'GDP_eva_df_oversampled.csv'
)

2020-05-26 00:09:22,214 Reading data from /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled
2020-05-26 00:09:22,216 Train: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_train_df_oversampled.csv
2020-05-26 00:09:22,218 Dev: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df_oversampled.csv
2020-05-26 00:09:22,220 Test: /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df_oversampled.csv


#### Second Stage fine-tuning

In [41]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-26 00:09:30,492 loading file /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotated_sample_for_training/best-model.pt


In [42]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-26 00:09:35,817 ----------------------------------------------------------------------------------------------------
2020-05-26 00:09:35,822 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [0.676527202129364,
  0.9050557613372803,
  0.3569127917289734,
  0.48473626375198364,
  0.9527653455734253,
  0.5228034257888794,
  0.33797508478164673,
  0.5376145243644714,
  0.24866335093975067,
  0.6115724444389343],
 'dev_score_history': [0.696969696969697,
  0.7171717171717171,
  0.7777777777777778,
  0.7575757575757576,
  0.6363636363636364,
  0.7777777777777778,
  0.8585858585858586,
  0.7171717171717171,
  0.8787878787878788,
  0.7777777777777778],
 'test_score': 0.8787878787878788,
 'train_loss_history': [1.248479413986206,
  0.993856155872345,
  0.6782370209693909,
  0.6985384285449981,
  0.45320078134536745,
  0.7802813291549683,
  0.33975547552108765,
  0.349251726269722,
  0.393967068195343,
  0.3563199996948242]}

In [0]:
# MACRO_AVG: acc 0.8787878787878788 - f1-score 0.8233082706766918

In [79]:
## load the 2nd-stage finetuned model:

finetuned_classifier = TextClassifier.load(new_data_folder + 'best-model.pt')

2020-05-26 01:29:23,110 loading file /content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/best-model.pt


In [81]:
# predict same sentence from above:

sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

finetuned_classifier.predict(sentence)

print(sentence.labels) ## correct

[-1 (0.885)]


In [64]:
dir(sentence.labels)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [74]:
dir(sentence.labels[0])


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_score',
 '_value',
 'score',
 'to_dict',
 'value']

In [75]:
sentence.labels[0].value

'0'

In [76]:
sentence.labels[0].score

0.9955407977104187

In [51]:
## error analysis

# get gold labels
#print(corpus.test[0])
print(corpus.test[0])
print(corpus.test[0].labels)



Sentence: "Trump claims victory as GDP grows at fastest pace since 2014 . President Donald Trump said the U.S. economy is on track to reach an annual growth rate of more than 3 per cent , as he celebrated a report Friday that the economy expanded in the second quarter at the fastest pace in four years ."   [− Tokens: 57  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (1.0)]


In [52]:
len(corpus.test)

33

In [0]:
gdp_test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/capstone_betterdwelling/annotations_bnn_cbc/oversampled/GDP_eva_df_oversampled.csv",usecols=['title_desc_sent_1','title_desc'])

In [57]:
gdp_test_df.head()

,title_desc_sent_1,title_desc
0,1,Trump claims victory as GDP grows at fastest p...
1,1,"Canada's economy grew 0.1% in August, Statisti..."
2,1,Canada’s economy grows faster than expected. F...
3,0,Why the Canadian economy seems divorced from t...
4,0,"The candidates: Great Slave. Patrick Scott, Ka..."


In [59]:
gdp_test_df['title_desc'].iloc[0]

'Trump claims victory as GDP grows at fastest pace since 2014. President Donald Trump said the U.S. economy is on track to reach an annual growth rate of more than 3 per cent, as he celebrated a report Friday that the economy expanded in the second quarter at the fastest pace in four years.'

In [82]:
correct = 0

for i in range(len(corpus.test)):
  print(corpus.test[i])
  

  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  sentence = Sentence(gdp_test_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  print(sentence.labels)

  #get gold label
  print(corpus.test[i].labels)

  #calculate correct guesses
  if sentence.labels[0].value == corpus.test[i].labels[0].value:
    correct += 1


Sentence: "Trump claims victory as GDP grows at fastest pace since 2014 . President Donald Trump said the U.S. economy is on track to reach an annual growth rate of more than 3 per cent , as he celebrated a report Friday that the economy expanded in the second quarter at the fastest pace in four years ."   [− Tokens: 57  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (0.9621)]
[1 (1.0)]
Sentence: "Canada 's economy grew 0.1 % in August , Statistics Canada reports . Gross domestic product edged up slightly , but fell short of expectations"   [− Tokens: 25  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (0.6064)]
[1 (1.0)]
Sentence: "Canada ’s economy grows faster than expected . Factories lead fastest economic growth in five months"   [− Tokens: 16  − Sentence-Labels: {'class': [1 (1.0)]}]
[1 (0.9683)]
[1 (1.0)]
Sentence: "Why the Canadian economy seems divorced from traditional signals : Don Pittis . With housing and oil off the boil , why has n't the Canadian economy gone into free fall ?" 

In [83]:
 correct #27/33 = 82% accuracy

 #reasonable wrongs

## words in negative sense but meaning is neutral
# Sentence: "Why the Canadian economy seems divorced from traditional signals : Don Pittis . With housing and oil off the boil , why has n't the Canadian economy gone into free fall ?"   [− Tokens: 32  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.9096)]
# [0 (1.0)]

## not learning enough that sentiment should be Canada-specific
# Sentence: "Trump is threatening Iran with more sanctions — but what 's left to target ? . Current sanctions have already sent the country spiralling into a deep recession"   [− Tokens: 28  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.9762)]
# [0 (1.0)]

## not learning enough that sentiment should be GDP-specific
# Sentence: "If you thought August was bad , get ready for the worst month for Canadian stocks . In the past 10 years , the TSX has dropped an average 1.5 % in September"   [− Tokens: 33  − Sentence-Labels: {'class': [0 (1.0)]}]
# [-1 (0.5721)]
# [0 (1.0)]

## not that reasonable wrongs -- all predicted to the opposite extreme... problematic!

## 'stabilized' overshadowed by 'slowing' and 'weakest'? -- quite inconfident prediction
# Sentence: "China 's economy grew 6 % in fourth quarter as demand stabilized . China ’s economy stabilized last quarter after slowing to the weakest pace in almost three decades , with the first acceleration in investment since June signaling that a firmer recovery could be underway ."   [− Tokens: 47  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.5108)]
# [1 (1.0)]

## 'fastest growth' vs "trade tension"/ "threat"
# Sentence: "Oil drives Canada 's fastest economic growth spurt in a year . Canada ’s economy grew at the fastest pace in a year , further evidence of a solid expansion even as trade tensions with the U.S. remain a threat ."   [− Tokens: 41  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.6974)]
# [1 (1.0)]

## 'boom','rosy' vs 'bumpy ride' -- description balanced out but positive headline should have more weight.
# Sentence: "Mining boom to drive economic growth in territories beyond rest of Canada : report . The Conference Board of Canada says outlook rosy for Nunavut and Yukon , but N.W.T. in for a bumpy ride"   [− Tokens: 35  − Sentence-Labels: {'class': [1 (1.0)]}]
# [-1 (0.5803)]
# [1 (1.0)]

27

In [0]:

## check accuracy if only count confident predictions(condifence score > 0.7?)
confident_total = 0
confident_correct = 0

for i in range(len(corpus.test)):
  #print(corpus.test[i])
  

  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  sentence = Sentence(gdp_test_df['title_desc'].iloc[i])

  finetuned_classifier.predict(sentence)

  #print(sentence.labels)

  #get gold label
  #print(corpus.test[i].labels)

  #calculate correct guesses
  if sentence.labels[0].score > 0.7:
    confident_total += 1
    if sentence.labels[0].value == corpus.test[i].labels[0].value:
      confident_correct += 1

In [86]:
confident_correct

24

In [87]:
confident_total

26

In [88]:
confident_correct/confident_total ## higher accuracy, pick only confident predictions for good visualization! also for correlation calculation?

0.9230769230769231